In [1]:
import json
from pprint import pprint
import sys
import unicodedata

import fiona
from shapely.geometry import shape, mapping
from shapely.ops import cascaded_union
from django.contrib.gis.geos import GEOSGeometry
from fiona.crs import to_string, from_epsg
from fiona.transform import transform_geom
from pyproj import Proj

from madmex.models import Region, Country
from madmex.util.spatial import feature_transform

In [2]:
#name_admin = 'MEX' or column_name = 'AGUACATE_DIANA'
name_admin = 'mex_simple'
#column_name = 'NAME_1' or column_name = ''
column_name = ''
#shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/MEX_adm_shp/MEX_adm1.shp' 
#or 
#shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/diana_poligonos_aguacate/poly_inventarios_buff30m/Inventario_1995_buff30m_poly.shp'
shapefile = '/LUSTRE/MADMEX/shapefiles/mexico_contorno_simple/mex_simple.shp'

In [3]:
with fiona.open(shapefile,
                encoding='utf-8') as src:
    fc = list(src)
    src_crs = src.crs
    to_string_crs = to_string(src_crs)
    proj_crs = Proj(src.crs)
    if not proj_crs.crs.is_geographic:
        fc_proj = [feature_transform(x,"+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",to_string_crs) for x in fc]
    else:
        fc_proj = fc

In [4]:
[feat['geometry']['type'] for feat in fc_proj][0:4]

['Polygon', 'MultiPolygon', 'MultiPolygon', 'MultiPolygon']

In [5]:
shape_list = [shape(feat['geometry']) for feat in fc_proj]

In [6]:
country_shape = cascaded_union(shape_list)

In [9]:
country, _ = Country.objects.get_or_create(the_geom=GEOSGeometry(country_shape.wkt),
                                       name=name_admin)

In [10]:
# Ingest features
for k in range(0,len(fc_proj)):
    shapefile = shape_list[k]
    geom = GEOSGeometry(shapefile.wkt,4326)
    
    if not column_name: #needs to generate name for every entry
        name = name_admin + '_%s'
        name_feature = name % fc_proj[k]['id']
    else:
        name_feature = fc_proj[k]['properties'][column_name].replace(" ", "_")
        name_feature_normalized = unicodedata.normalize('NFKD', name_feature).encode('ASCII', 'ignore').decode('utf-8')
        name_feature = name_admin + '_' + name_feature_normalized
    _ = Region.objects.get_or_create(name=name_feature,
                                     the_geom=geom,
                                     country=country)
    print(name_feature)

mex_simple_0


In [12]:
region = 'mex_simple_0'
region_geom = Region.objects.get(name=region).the_geom
region_geom

In [13]:
region_geom.wkt

'POLYGON ((-92.57917209239469 21.02663221276538, -90.90110566771392 22.73725600585619, -88.714599609375 22.5734382645724, -86.209716796875 22.26368048235324, -86.23168945312499 20.77665905187882, -87.0831298828125 19.85456068070104, -87.20947265625 17.34539508915956, -87.8741455078125 16.63619187839766, -88.5662841796875 17.67019390058734, -90.7965087890625 17.54453427009908, -90.802001953125 17.114542657152, -90.252685546875 16.55722718710274, -90.2691650390625 15.8186809628796, -91.6094970703125 15.86095760626048, -91.9171142578125 15.15697371337767, -92.0050048828125 14.49850814944622, -92.3895263671875 14.08862930844272, -93.636474609375 15.40602366459229, -94.9493408203125 15.81339576046059, -96.6522216796875 15.07742767484799, -103.7054443359375 18.04142122189195, -105.018310546875 18.87510275035649, -106.01806640625 20.3034175184893, -107.24853515625 21.40704812346109, -107.138671875 23.36747130375967, -109.324951171875 22.66470981017683, -110.533447265625 18.37537909403182, -11

In [ ]:
country = 'mex_simple'
country_geom = Country.objects.get(name=country).the_geom
country_geom